# Examen

En este notebook de Jupyter haremos los gráficos que posteriormente usaremos para la entrega final en forma de tríptico. El tema a investigar a través de representaciones gráficas generadas por computador es el de como se puede generar rankings en la promoción de artes marciales mixtas Ultimate Fighting Championship (UFC). Para ver esto se creó un método de ranking llamado HD y se comparó contra una base de datos que contiene los valores de los puntajes de ranking "entrega-final/nkshv/ELO/ELO-scraper/src/test.py"